In [15]:
code_dir <- getwd()
tool_dir <- unlist(strsplit(code_dir,"/"))
tool_dir <- head(tool_dir, length(tool_dir)-2)
tool_dir <- paste(c(tool_dir, "tools"), collapse="/")
source(file.path(tool_dir, "function_GSEA.R"))
base_dir <- gsub("/codes_local", "", code_dir)

In [18]:
base_dir

[1] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/0_Acute-Chronic"

## Resampled Louvain cluster GSEA

In [20]:
##########----- Parameters-----##########
analysis_name <- c("0", "resampled")
#########################################
'/media/pipkin/ROCKET-PRO/CD8_DEV_SC/0_Acute-Chronic/1_Scanpy/0_Scanpy_out_resampled/1_avg_expr/louvain_mean_scaled_expr_z.csv'
sum.dir <- file.path(base_dir, '1_Scanpy', 
                     paste(analysis_name[1], "Scanpy_out", analysis_name[2], sep="_"),
                    "3_GSEA")
dir.create(sum.dir, showWarnings = FALSE, recursive=TRUE)
setwd(sum.dir)

[1] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/0_Acute-Chronic/1_Scanpy/0_Scanpy_out_resampled/1_avg_expr/louvain_mean_scaled_expr_z.csv"

[1] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/0_Acute-Chronic/1_Scanpy/0_Scanpy_out_resampled/3_GSEA"

In [ ]:


###----- Select gene signature
gs.dir <- file.path(base_dir, 'z_References/9_Formatted_csv')
gs.files <- list.files(gs.dir, pattern='hs_sigs.csv', full.names=TRUE)

### 1.2 Per cluster seperate cell type

In [12]:
wk.dir <- '/media/pipkin/Yolanda/Exp391_Acute-Chronic_SC/1_1_SCANPY_PAGA/all--numSlt-rmWTNAV/3_GSEA/sep_typeLouvain'
setwd(wk.dir)

z.file <- '/media/pipkin/Yolanda/Exp391_Acute-Chronic_SC/1_1_SCANPY_PAGA/all--numSlt-rmWTNAV/1_norm_counts/all_norm_counts_named_c10_nbPctl_Z_naOmit--allNumSltrmWTNAV_typeLouvain_z.csv'
z.tb <- read_csv(z.file)

gs.all.file <- '/media/pipkin/Yolanda/Exp391_Acute-Chronic_SC/z_References/GSEA/Combined_GEO_GSEA_signatures.csv'

for (i in tail(colnames(z.tb), -1)){
    useGroup <- i
    outName <- useGroup
    GSEA_analysis(z.file, useGroup, outName, gs.all.file, 1)
}

Parsed with column specification:
cols(
  .default = col_double(),
  gene_name = col_character()
)

See spec(...) for full column specifications.

Parsed with column specification:
cols(
  .default = col_double(),
  gene_name = col_character()
)

See spec(...) for full column specifications.

Parsed with column specification:
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)

preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...

Warning message:
“Removed 2 rows containing missing values (geom_rect).”
Warning message:
“Removed 1 rows containing missing values (geom_rect).”
Warning message:
“Removed 2 rows containing missing values (geom_rect).”
Warning message:
“Removed 1 rows containing missing values (geom_rect).”
Parsed with column specification:
cols(
  .default = col_double(),
  gene_name = col_character()
)

See spec(...) for full column specific

Warning message:
“Removed 2 rows containing missing values (geom_rect).”
Warning message:
“Removed 2 rows containing missing values (geom_rect).”
Parsed with column specification:
cols(
  .default = col_double(),
  gene_name = col_character()
)

See spec(...) for full column specifications.

Parsed with column specification:
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)

preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...

Warning message:
“Removed 1 rows containing missing values (geom_rect).”
Warning message:
“Removed 2 rows containing missing values (geom_rect).”
Warning message:
“Removed 2 rows containing missing values (geom_rect).”
Warning message:
“Removed 1 rows containing missing values (geom_rect).”
Parsed with column specification:
cols(
  .default = col_double(),
  gene_name = col_character()
)

See spec(...) for full column specifica

Warning message:
“Removed 1 rows containing missing values (geom_rect).”
Warning message:
“Removed 2 rows containing missing values (geom_rect).”
